<a href="https://colab.research.google.com/github/varshneyanushka/IITI_SOC_ML9/blob/main/mahabharat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer, AutoModelForSeq2SeqLM, pipeline

# Following model is fine-tuned by team using pretained t5-small
model_id = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained("Shreeyut/results")
model_2 = AutoModelForSeq2SeqLM.from_pretrained("Shreeyut/grammar_expert_1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
#Folllowing functions are defined to give instructions to the model to perform specific task on a single sentence.
def explainer(input_text):
  text = f'{input_text}'
  input_ids = tokenizer(text,return_tensors="pt").input_ids
  outputs = model_2.generate(input_ids,max_length=256)
  edited_text = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return edited_text

def generate_correct_grammar_sentence(input_text):
  text = f'Fix grammatical errors in this sentence: {input_text}'
  input_ids = tokenizer(text,return_tensors="pt").input_ids
  outputs = model.generate(input_ids,max_length=256)
  edited_text = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return edited_text

def paraphrase_text(input_text):
  text = f'Paraphrase this: {input_text}'
  input_ids = tokenizer(text,return_tensors="pt").input_ids
  outputs = model.generate(input_ids,max_length=256)
  paraphrased_text = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return paraphrased_text

def improve_grammar(input_text):
  text = f'Improve the grammaticality:{input_text}'
  input_ids = tokenizer(text,return_tensors='pt').input_ids
  outputs = model.generate(input_ids,max_length=256)
  improved_grammar = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return improved_grammar

def simplification(input_text):
  text = f'Simplify this text:{input_text}'
  input_ids = tokenizer(text,return_tensors='pt').input_ids
  outputs = model.generate(input_ids,max_length = 256)
  simplified_grammar = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return simplified_grammar

def cohesive(input_text):
  text = f'Fix coherence in this text:{input_text}'
  input_ids = tokenizer(text,return_tensors='pt').input_ids
  outputs = model.generate(input_ids,max_length = 256)
  cohesive_grammar = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return cohesive_grammar


def Neutralize(input_text):
  text = f'Make this text more neutral:{input_text}'
  input_ids = tokenizer(text,return_tensors='pt').input_ids
  outputs = model.generate(input_ids,max_length = 256)
  Neutralized_grammar = tokenizer.decode(outputs[0],skip_special_tokens=True)
  return Neutralized_grammar

In [ ]:
#Following functions are defined to break paragraph into sentences and then applying above functions on them and appending the transformed sentences into a paragraph.
def correct_paragraph(paragraph):
  wrong_english =[]
  para = paragraph
  doc = nlp(para)
  for sentence in doc.sents:
    wrong_english.append(sentence)
  corrected_english = []
  paraphrased_text = []
  for sentence in wrong_english:
    corrected_english.append(generate_correct_grammar_sentence(str(sentence)))
  corrected_paragraph  = ' '.join(corrected_english)
  for sentence in corrected_english:
    paraphrased_sentence = paraphrase_text(sentence)
    paraphrased_text.append(paraphrased_sentence)
  paraphrased_paragragh = ' '.join(paraphrased_text)
  return corrected_paragraph,paraphrased_paragragh


def explaining_paragraph(paragraph):
  english = []
  para = paragraph
  doc = nlp(para)
  for sentence in doc.sents:
    english.append(sentence)
  explain_english = []
  for sentence in english:
    explain_english.append(explainer(str(sentence)))
  explain_paragraph = ' '.join(explain_english)
  return explain_paragraph

def Neutralize_paragraph(paragraph):
  english = []
  para = paragraph
  doc = nlp(para)
  for sentence in doc.sents:
    english.append(sentence)
  neutralized_english = []
  for sentence in english:
    neutralized_english.append(Neutralize(str(sentence)))
  neutralized_paragraph = ' '.join(neutralized_english)
  return neutralized_paragraph


def Cohesive_paragraph(paragraph):
  english = []
  para = paragraph
  doc = nlp(para)
  for sentence in doc.sents:
    english.append(sentence)
  cohesive_english = []
  for sentence in english:
    cohesive_english.append(cohesive(str(sentence)))
  cohesive_paragraph = ' '.join(cohesive_english)
  return cohesive_paragraph

def Simplified_paragraph(paragraph):

  english = []
  para = paragraph
  doc = nlp(para)
  for sentence in doc.sents:
    english.append(sentence)
  simplified_english = []
  for sentence in english:
    simplified_english.append(simplification(str(sentence)))
  simplified_paragraph = ' '.join(simplified_english)
  return simplified_paragraph

In [ ]:
!pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import gradio as gr

In [ ]:
!pip  install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 16.3 MB/s eta 0:00:00


In [ ]:
import speech_recognition as sr

In [ ]:
#Following function is defined to take the input and give the desired output
def process_paragraph(paragraph,process_type,grammatical_error):
        corrected_paragraph, paraphrased_paragraph = correct_paragraph(paragraph)
        explained_paragraph = explaining_paragraph(paragraph)
        if process_type == "Neutralize" and grammatical_error == True :
                neutralized_paragraph = Neutralize_paragraph(corrected_paragraph)
                return f"Corrected Paragraph:\n{corrected_paragraph}\n\nParaphrased Paragraph:\n{paraphrased_paragraph}\n\nNeutralized Paragraph:\n{neutralized_paragraph}\n\nErrors in Paragraph:\n{explained_paragraph}"
        elif process_type == "Cohesive" and grammatical_error == True:
                cohesive_paragraph = Cohesive_paragraph(corrected_paragraph)
                return f"Corrected Paragraph:\n{corrected_paragraph}\n\nParaphrased Paragraph:\n{paraphrased_paragraph}\n\nCohesive Paragraph:\n{cohesive_paragraph}\n\nErrors in Paragraph:\n{explained_paragraph}"
        elif process_type == "Simplify" and grammatical_error == True:
                simplified_paragraph = Simplified_paragraph(corrected_paragraph)
                return f"Corrected Paragraph:\n{corrected_paragraph}\n\nParaphrased Paragraph:\n{paraphrased_paragraph}\n\nSimplified Paragraph:\n{simplified_paragraph}\n\nErrors in Paragraph:\n{explained_paragraph}"
        elif process_type == "Neutralize" and grammatical_error == False  :
                neutralized_paragraph = Neutralize_paragraph(corrected_paragraph)
                return f"Corrected Paragraph:\n{corrected_paragraph}\n\nParaphrased Paragraph:\n{paraphrased_paragraph}\n\nNeutralized Paragraph:\n{neutralized_paragraph}"
        elif process_type == "Cohesive" and grammatical_error == False :
                cohesive_paragraph = Cohesive_paragraph(corrected_paragraph)
                return f"Corrected Paragraph:\n{corrected_paragraph}\n\nParaphrased Paragraph:\n{paraphrased_paragraph}\n\nCohesive Paragraph:\n{cohesive_paragraph}"
        elif process_type == "Simplify" and grammatical_error == False :
                simplified_paragraph = Simplified_paragraph(corrected_paragraph)
                return f"Corrected Paragraph:\n{corrected_paragraph}\n\nParaphrased Paragraph:\n{paraphrased_paragraph}\n\nSimplified Paragraph:\n{simplified_paragraph}"

In [ ]:
# Creating a Gradio interface
interface=gr.Interface(process_paragraph,
                      inputs=[gr.Textbox(max_lines=20,label="Text Input", placeholder="Enter text here...",),
                              gr.Dropdown([ "Neutralize", "Cohesive", "Simplify"],label="Select Processing Type",info="Select what you want to do with your text..."
                               ),
                               gr.Checkbox(label="Grammatical Error",info="Select to see the error in text")],
                       outputs=[gr.Textbox()],
                       theme="glass",
                       examples=[["Guy Leech is still a celebrity today. Guy Leech regularly appears on television as an advocate for health and fitness.","Cohesive","Explain"],
                                 ["There was a girl standing there, staring into the room.","Simplify","Explain"],
                                 ["""chloroform "the molecular lifesaver" an article at oxford university providing interesting facts about chloroform.""","Neutralize","Explain"],
                                 ["melina is currently dating john cena, known as johnny nitro.","Neutralize","Explain"],
                                 ["Injury, bad luck and inconsistency prevented him from ever winning the series. He was certainly good enough.","Cohesive","Explain"],
                                 ["That's about 1.9 trillion by today's standards.","Simplify","Explain"]],
                       description="""How to Use:

                                  <font size="4">1</font>.Enter Your Paragraph: Type or paste the paragraph you want to process into the text box provided.,
                                 <font size="4">2</font>. Select Processing Type: Choose the type of processing you want to apply from the dropdown menu. The options include:
                                      (a)<U>Correct and Paraphrase</U>: Automatically corrects grammatical errors in your paragraph and provides a paraphrased version.
                                      (b)<U>Neutralize</U>: Adjusts the tone of your paragraph to be more neutral.
                                      (c)<U>Cohesive</U>: Enhances the coherence of your paragraph, making it flow better.
                                      (d)<U>Simplify</U>: Simplifies complex sentences for better readability and understanding.,
                                  <font size="4">3</font>.Process and View Results: Click the submit button to process your paragraph. The results will be displayed, showing the refined version of your text.""",
                       title="Paragraph Processor"
                       )
interface.launch(inbrowser=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1991ae30552104c350.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# def process_input(text_input, audio_input, process_type):
#     if audio_input:
#         text_input = audio_to_text(audio_input)

#     corrected_paragraph, paraphrased_paragraph = correct_paragraph(text_input)
#     processed_output = process_paragraph(corrected_paragraph, process_type)
#     return text_input, corrected_paragraph,processed_output

In [ ]:
# with gr.Blocks() as interfac:
#     gr.Markdown(
#         """
#         ## Audio Transcription, Grammar Correction, and Text Processing
#         This application allows you to transcribe audio, correct grammar, and process text to enhance coherence, simplify, neutralize, and paraphrase.
#         """
#     )

#     with gr.Row():
#         text_input = gr.Textbox(label="Text Input", placeholder="Enter text here...", lines=5)
#         audio_input = gr.Audio(label="Audio Input", type="filepath")

#     process_type = gr.Dropdown([ "Neutralize", "Cohesive", "Simplify"], label="Select Processing Type")

#     transcribed_text = gr.Textbox(label="Transcribed Text")
#     corrected_paragraph = gr.Textbox(label="Corrected Sentence")
#     processed_output = gr.Textbox(label="Processed Output")

#     process_button = gr.Button("Process")

#     process_button.click(
#         process_input,
#         inputs=[text_input, audio_input, process_type],
#         outputs=[transcribed_text, corrected_paragraph, processed_output]
#     )

# interfac.launch(share=True)